# FCC Application Scraping

## Set up selenium webdriver with Firefox
* Download Firefox driver [geckodriver](https://github.com/mozilla/geckodriver/releases) for selenium
* Move the driver executable into the same directory as this notebook (or update the path to the executable in `scrape_table_for_call_sign`)
* `pip install selenium`

In [1]:
from bs4 import BeautifulSoup
import time
import random
from math import sqrt
import csv

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options  
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException        

In [3]:
import os
import pandas as pd

## Define functions for scraping table for an individual call sign

In [4]:
def element_exists(driver, xpath):
    """
    Helper function that checks if the element at the given xpath exists on
    driver's current page. This is used to check if there is a "Next" button
    -- to scrape through multiple pages of the FCC's paginated tables.
    """
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


In [5]:
def extract_callsign(innerHTML):
    '''
    Helper function that extracts the callsign from Application search results page.
    This is used to append the CALLSIGN to each row when scraping table data
    '''
    soup = BeautifulSoup(innerHTML, "lxml")
    
    # tag attributes for element containing text "Call Sign = " 
    tag_attrs = {"class":"cell-pri-light", "valign":"bottom", "align":"left"}
    
    # extract callsign as text
    callsign = (
        soup.find_all(attrs=tag_attrs, limit=1)[0]
        .find_all("b")[0]
        .get_text()
    )
    
    return callsign

In [6]:
def count_results(innerHTML):
    '''
    Helper function that extracts the total number of results across all pages.
    This is used to check the scraped table data has the expected number of rows.
    '''
    
    soup = BeautifulSoup(innerHTML, "lxml")
    
    # tag attributes for element containing text "Matches 1-47 (of 47)"
    tag_attrs = {"class":"cell-pri-dark", "width":"35%", "valign":"middle", "align":"left"}

    # extract results count from inside "(of 47)" as integer 47
    count = int(soup.find_all(attrs=tag_attrs, limit=1)[0]
                .find_all("b")[1]
                .get_text()
               )
    
    return count

In [7]:
def scrape_table_page(innerHTML):
    """
    Extracts the following details from a call sign table page:
        * File Number
        * Call Sign/Lease ID
        * Applicant Name
        * FRN
        * Purpose - split into Purpose_Main and Purpose_Note
        * Radio
        * Service
        * Receipt
        * Date
        * Status
    Returns as a list of lists.
    """
    
    # Extract search data from innerHTML soup
    soup = BeautifulSoup(innerHTML, "lxml")

    CALLSIGN = extract_callsign(innerHTML)
    table_page = soup.find("table", {"summary": "Application search results"})
    
    # Extract search results as list of rows
    rows = table_page.find_all("tr")[1:-1] # note: skip header and pagination rows
    
    # Extract results by row into table, attaching callsign number
    table_data = []
    
    for row in rows:  
        
        row_data = [CALLSIGN, ]
        cols = row.find_all("td")
        
        # extract values from each cell in row
        for i in range(1, len(cols)): # note: skip the first (index) cell
            
            # extract cell data, trim leading/trailing whitespace
            cell_data = cols[i].get_text().strip()
            
            # fifth-index cells "Purpose" has to get split into two cols
            if i == 5:
                
                purpose = cell_data.split("\n")
                purpose_main = purpose[0].strip()
                purpose_note = ""
                
                # sometimes the "Purpose" column has an optional note
                if len(purpose) >= 2:
                    purpose_note = purpose[-1].strip()
                    
                row_data.append(purpose_main)
                row_data.append(purpose_note)
            
            # other cells can be inserted as normal
            else:
                row_data.append(cell_data)
                
        table_data.append(row_data)
    
    return(table_data)


In [8]:
def cache_to_csv(lst, path):

    no_rows = len(lst)
    
    with open(path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(lst)

    with open(path, 'r', newline='') as f:
        reader = csv.reader(f)
        check = list(reader)

    assert(len(lst) == len(check))
    
    print('LOG[INFO]: Wrote %s rows to %s' % (no_rows, path))

In [9]:
def make_html_path(parent_dir, call_sign, page_no):
    '''
    Helper function that generates path for caching html pages
    '''
    path = ("%s/%s-%s.html" % (parent_dir, call_sign, str(page_no).zfill(3)))
    return path

In [10]:
def scrape_tables_for_call_sign(call_sign, html_cache, csv_cache, without_browser=False):
    """
    Scrapes the FCC Application Search database for a given call sign.
    
    Caches results pages as HTML
    
    Returns the table as a list of lists, for example:
    [
      ['File Number', 'Call Sign/Lease ID', 'Applicant Name', 'FRN', 'Purpose_Main', 'Purpose_Note', 'Radio Service', 'Receipt Date', 'Status'],
      ['0000214527',  'KNKN555',            'AT&T',           '',    'Amendment',    'Transfer Control', 'AL',        '09/05/2000',   'Granted'],
      ...,
      ...
    ]
    
    """
    tick = time.time()
    print("LOG[INFO]: START %s " % call_sign)
    
    
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")
    
    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp")
    
    # specify the call sign that we want to scrape for
    callsign = firefox.find_element_by_id("ulsCallSign")
    callsign.send_keys(call_sign)
    
    # set pagination to 100 (maximum)
    pagination = Select(firefox.find_element_by_xpath("//select[@name='pageSize']"))
    pagination.select_by_visible_text("100")
    
    # submit the search form, wait for load
    # NOTE: find form submission by form element name, not search button image
    firefox.find_element_by_name("search").submit()
    WebDriverWait(firefox, 45).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 45).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
    
    # extract and cache html from first page
    page = 1
    innerHTML = firefox.execute_script("return document.body.innerHTML")
    with open(make_html_path(html_cache, call_sign, page), 'w') as f:
        f.write(innerHTML)
    
    # store count of search results for later check
    no_results = count_results(innerHTML)
    
    # Scrape results data from first page
    search_data = []
    search_data += scrape_table_page(innerHTML)
    
    # as long there is a link to "next page"...
    next_page_xpath = "//*[@title='Next page of results']"
    while element_exists(firefox, next_page_xpath):
        # ... go to next page
        firefox.find_element_by_xpath(next_page_xpath).click()
        # ... wait for load
        WebDriverWait(firefox, 45).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
        # .. increment page counter, extract and cache html 
        page += 1
        innerHTML = firefox.execute_script("return document.body.innerHTML")
        with open(make_html_path(html_cache, call_sign, page), 'w') as f:
            f.write(innerHTML)
        # ... scrape and add page results to all results
        search_data += scrape_table_page(innerHTML)

    # gracefully close the webdriver
    firefox.quit()
    
    # Check that all results have been scraped
    no_rows = len(search_data)
    #assert(no_rows == no_results)
    print("LOG[INFO]: Scraped %s rows out of %s results" % (no_rows, no_results))

    # Cache search results table to csv (all results from all pages)
    csv_path = ''.join([csv_cache, '/', call_sign, ".csv"])
    cache_to_csv(search_data, csv_path)
    

    tock = time.time()
    print("LOG[INFO]: END %s (%2f seconds)" % (call_sign, tock - tick))
    
    return search_data
     

## Scrape the tables for a list of call signs (loop)

In [11]:
# read in list of all callsigns from file

callsign_path = 'data-cache/cma_callsign_lookup.csv'
callsigns = pd.read_csv(callsign_path).sort_values(by=['callsign_leaseID'])['callsign_leaseID'].tolist()

In [14]:
# Set paths for caching

html_cache = 'search-cache-v2/html'
csv_cache = 'search-cache-v2/csv'

In [15]:
## Attempt 6 -- WEEKEND

tick = time.time()
print("LOG[INFO]: START TIME is %s\n" % (time.strftime("%I:%M %p")))

tries = 0
max_tries = 100

start_index = 1 # skip license = 'B'
end_index = len(callsigns)

error_locs = []

for cs in callsigns[start_index:end_index]:    
    if tries < max_tries:
        try:
            scrape_tables_for_call_sign(cs, html_cache, csv_cache, without_browser=True)
            pause = (random.randint(1,30))
            time.sleep(pause)
        except Exception as e:
            tries += 1
            error_index = callsigns.index(cs)
            error_locs.append(error_index)
            print("Error at %s, located at callsigns[%s]" % (cs, error_index))
            print("Error:", e)
            pause = (random.randint(420,600) * min(sqrt(tries), 6)) # max wait is 1hr (3600secs)
            time.sleep(pause)
        finally:
            end_index = callsigns.index(cs)
            pass
    else:
        print("LOG[INFO]: Maximum number of errors (%s) reached." % (tries))
        break

        
tock = time.time()


print("LOG[INFO]: END TIME is %s (TOTAL: %2f hours)" 
      % (time.strftime("%I:%M %p"), ((tock - tick)/60/60))
     )
print("Completed searches from %s [%s] to %s [%s]." 
      % (callsigns[start_index], start_index, cs, end_index)
     )
print("Errors at:", error_locs)

error_locs

LOG[INFO]: START TIME is 05:38 PM

LOG[INFO]: START KNKA200 
LOG[INFO]: Scraped 266 rows out of 266 results
LOG[INFO]: Wrote 266 rows to search-cache-v2/csv/KNKA200.csv
LOG[INFO]: END KNKA200 (16.417499 seconds)
LOG[INFO]: START KNKA201 
LOG[INFO]: Scraped 204 rows out of 204 results
LOG[INFO]: Wrote 204 rows to search-cache-v2/csv/KNKA201.csv
LOG[INFO]: END KNKA201 (14.480333 seconds)
LOG[INFO]: START KNKA202 
LOG[INFO]: Scraped 133 rows out of 133 results
LOG[INFO]: Wrote 133 rows to search-cache-v2/csv/KNKA202.csv
LOG[INFO]: END KNKA202 (13.893886 seconds)
LOG[INFO]: START KNKA203 
LOG[INFO]: Scraped 151 rows out of 151 results
LOG[INFO]: Wrote 151 rows to search-cache-v2/csv/KNKA203.csv
LOG[INFO]: END KNKA203 (15.324007 seconds)
LOG[INFO]: START KNKA204 
LOG[INFO]: Scraped 171 rows out of 171 results
LOG[INFO]: Wrote 171 rows to search-cache-v2/csv/KNKA204.csv
LOG[INFO]: END KNKA204 (13.617981 seconds)
LOG[INFO]: START KNKA205 
LOG[INFO]: Scraped 294 rows out of 294 results
LOG[INF

LOG[INFO]: START KNKA248 
LOG[INFO]: Scraped 207 rows out of 207 results
LOG[INFO]: Wrote 207 rows to search-cache-v2/csv/KNKA248.csv
LOG[INFO]: END KNKA248 (16.961281 seconds)
LOG[INFO]: START KNKA249 
LOG[INFO]: Scraped 128 rows out of 128 results
LOG[INFO]: Wrote 128 rows to search-cache-v2/csv/KNKA249.csv
LOG[INFO]: END KNKA249 (16.420207 seconds)
LOG[INFO]: START KNKA250 
LOG[INFO]: Scraped 164 rows out of 164 results
LOG[INFO]: Wrote 164 rows to search-cache-v2/csv/KNKA250.csv
LOG[INFO]: END KNKA250 (14.316145 seconds)
LOG[INFO]: START KNKA251 
LOG[INFO]: Scraped 60 rows out of 60 results
LOG[INFO]: Wrote 60 rows to search-cache-v2/csv/KNKA251.csv
LOG[INFO]: END KNKA251 (14.788595 seconds)
LOG[INFO]: START KNKA252 
LOG[INFO]: Scraped 79 rows out of 79 results
LOG[INFO]: Wrote 79 rows to search-cache-v2/csv/KNKA252.csv
LOG[INFO]: END KNKA252 (12.351965 seconds)
LOG[INFO]: START KNKA253 
LOG[INFO]: Scraped 132 rows out of 132 results
LOG[INFO]: Wrote 132 rows to search-cache-v2/csv

LOG[INFO]: START KNKA295 
LOG[INFO]: Scraped 107 rows out of 107 results
LOG[INFO]: Wrote 107 rows to search-cache-v2/csv/KNKA295.csv
LOG[INFO]: END KNKA295 (16.894886 seconds)
LOG[INFO]: START KNKA296 
LOG[INFO]: Scraped 135 rows out of 135 results
LOG[INFO]: Wrote 135 rows to search-cache-v2/csv/KNKA296.csv
LOG[INFO]: END KNKA296 (16.423663 seconds)
LOG[INFO]: START KNKA297 
LOG[INFO]: Scraped 124 rows out of 124 results
LOG[INFO]: Wrote 124 rows to search-cache-v2/csv/KNKA297.csv
LOG[INFO]: END KNKA297 (17.017123 seconds)
LOG[INFO]: START KNKA298 
LOG[INFO]: Scraped 36 rows out of 36 results
LOG[INFO]: Wrote 36 rows to search-cache-v2/csv/KNKA298.csv
LOG[INFO]: END KNKA298 (15.516844 seconds)
LOG[INFO]: START KNKA299 
LOG[INFO]: Scraped 92 rows out of 92 results
LOG[INFO]: Wrote 92 rows to search-cache-v2/csv/KNKA299.csv
LOG[INFO]: END KNKA299 (15.511608 seconds)
LOG[INFO]: START KNKA300 
LOG[INFO]: Scraped 68 rows out of 68 results
LOG[INFO]: Wrote 68 rows to search-cache-v2/csv/KN

LOG[INFO]: START KNKA342 
LOG[INFO]: Scraped 118 rows out of 118 results
LOG[INFO]: Wrote 118 rows to search-cache-v2/csv/KNKA342.csv
LOG[INFO]: END KNKA342 (16.953549 seconds)
LOG[INFO]: START KNKA343 
LOG[INFO]: Scraped 144 rows out of 144 results
LOG[INFO]: Wrote 144 rows to search-cache-v2/csv/KNKA343.csv
LOG[INFO]: END KNKA343 (18.955689 seconds)
LOG[INFO]: START KNKA344 
LOG[INFO]: Scraped 56 rows out of 56 results
LOG[INFO]: Wrote 56 rows to search-cache-v2/csv/KNKA344.csv
LOG[INFO]: END KNKA344 (17.108029 seconds)
LOG[INFO]: START KNKA345 
LOG[INFO]: Scraped 80 rows out of 80 results
LOG[INFO]: Wrote 80 rows to search-cache-v2/csv/KNKA345.csv
LOG[INFO]: END KNKA345 (17.285387 seconds)
LOG[INFO]: START KNKA346 
LOG[INFO]: Scraped 94 rows out of 94 results
LOG[INFO]: Wrote 94 rows to search-cache-v2/csv/KNKA346.csv
LOG[INFO]: END KNKA346 (18.273415 seconds)
LOG[INFO]: START KNKA347 
LOG[INFO]: Scraped 123 rows out of 123 results
LOG[INFO]: Wrote 123 rows to search-cache-v2/csv/KN

LOG[INFO]: START KNKA390 
LOG[INFO]: Scraped 100 rows out of 100 results
LOG[INFO]: Wrote 100 rows to search-cache-v2/csv/KNKA390.csv
LOG[INFO]: END KNKA390 (19.582047 seconds)
LOG[INFO]: START KNKA391 
LOG[INFO]: Scraped 85 rows out of 85 results
LOG[INFO]: Wrote 85 rows to search-cache-v2/csv/KNKA391.csv
LOG[INFO]: END KNKA391 (20.651694 seconds)
LOG[INFO]: START KNKA392 
LOG[INFO]: Scraped 70 rows out of 70 results
LOG[INFO]: Wrote 70 rows to search-cache-v2/csv/KNKA392.csv
LOG[INFO]: END KNKA392 (15.852944 seconds)
LOG[INFO]: START KNKA393 
LOG[INFO]: Scraped 75 rows out of 75 results
LOG[INFO]: Wrote 75 rows to search-cache-v2/csv/KNKA393.csv
LOG[INFO]: END KNKA393 (16.525636 seconds)
LOG[INFO]: START KNKA394 
LOG[INFO]: Scraped 70 rows out of 70 results
LOG[INFO]: Wrote 70 rows to search-cache-v2/csv/KNKA394.csv
LOG[INFO]: END KNKA394 (18.171138 seconds)
LOG[INFO]: START KNKA395 
LOG[INFO]: Scraped 61 rows out of 61 results
LOG[INFO]: Wrote 61 rows to search-cache-v2/csv/KNKA395.

...

LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache-v2/csv/WPQZ339.csv
LOG[INFO]: END WPQZ339 (14.531792 seconds)
LOG[INFO]: START WPQZ671 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache-v2/csv/WPQZ671.csv
LOG[INFO]: END WPQZ671 (16.185624 seconds)
LOG[INFO]: START WPRS845 
LOG[INFO]: Scraped 28 rows out of 28 results
LOG[INFO]: Wrote 28 rows to search-cache-v2/csv/WPRS845.csv
LOG[INFO]: END WPRS845 (17.017730 seconds)
LOG[INFO]: START WPRS901 
LOG[INFO]: Scraped 19 rows out of 19 results
LOG[INFO]: Wrote 19 rows to search-cache-v2/csv/WPRS901.csv
LOG[INFO]: END WPRS901 (14.430041 seconds)
LOG[INFO]: START WPRS917 
LOG[INFO]: Scraped 26 rows out of 26 results
LOG[INFO]: Wrote 26 rows to search-cache-v2/csv/WPRS917.csv
LOG[INFO]: END WPRS917 (14.302038 seconds)
LOG[INFO]: START WPRS922 
LOG[INFO]: Scraped 21 rows out of 21 results
LOG[INFO]: Wrote 21 rows to search-cache-v2/csv/WPRS922.csv
LOG[INFO]: END WPRS922 (14.0605

LOG[INFO]: START WPUN211 
LOG[INFO]: Scraped 3 rows out of 3 results
LOG[INFO]: Wrote 3 rows to search-cache-v2/csv/WPUN211.csv
LOG[INFO]: END WPUN211 (11.937567 seconds)
LOG[INFO]: START WPUP317 
LOG[INFO]: Scraped 33 rows out of 33 results
LOG[INFO]: Wrote 33 rows to search-cache-v2/csv/WPUP317.csv
LOG[INFO]: END WPUP317 (14.976414 seconds)
LOG[INFO]: START WPUP627 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache-v2/csv/WPUP627.csv
LOG[INFO]: END WPUP627 (15.098385 seconds)
LOG[INFO]: START WPUP631 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache-v2/csv/WPUP631.csv
LOG[INFO]: END WPUP631 (14.131257 seconds)
LOG[INFO]: START WPUX427 
LOG[INFO]: Scraped 19 rows out of 19 results
LOG[INFO]: Wrote 19 rows to search-cache-v2/csv/WPUX427.csv
LOG[INFO]: END WPUX427 (18.004602 seconds)
LOG[INFO]: START WPUY962 
LOG[INFO]: Scraped 5 rows out of 5 results
LOG[INFO]: Wrote 5 rows to search-cache-v2/csv/WPUY962.csv
LOG[INFO]: 

LOG[INFO]: START WQJE415 
LOG[INFO]: Scraped 20 rows out of 20 results
LOG[INFO]: Wrote 20 rows to search-cache-v2/csv/WQJE415.csv
LOG[INFO]: END WQJE415 (15.740812 seconds)
LOG[INFO]: START WQJL936 
LOG[INFO]: Scraped 22 rows out of 22 results
LOG[INFO]: Wrote 22 rows to search-cache-v2/csv/WQJL936.csv
LOG[INFO]: END WQJL936 (13.907814 seconds)
LOG[INFO]: START WQJS956 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache-v2/csv/WQJS956.csv
LOG[INFO]: END WQJS956 (14.738028 seconds)
LOG[INFO]: START WQKK274 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache-v2/csv/WQKK274.csv
LOG[INFO]: END WQKK274 (13.739608 seconds)
LOG[INFO]: START WQKM495 
LOG[INFO]: Scraped 7 rows out of 7 results
LOG[INFO]: Wrote 7 rows to search-cache-v2/csv/WQKM495.csv
LOG[INFO]: END WQKM495 (14.739037 seconds)
LOG[INFO]: START WQKQ560 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache-v2/csv/WQKQ560.csv
LOG[INFO]: 

LOG[INFO]: START WQVZ380 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache-v2/csv/WQVZ380.csv
LOG[INFO]: END WQVZ380 (14.319225 seconds)
LOG[INFO]: START WQWD617 
LOG[INFO]: Scraped 9 rows out of 9 results
LOG[INFO]: Wrote 9 rows to search-cache-v2/csv/WQWD617.csv
LOG[INFO]: END WQWD617 (14.208410 seconds)
LOG[INFO]: START WQWD618 
LOG[INFO]: Scraped 9 rows out of 9 results
LOG[INFO]: Wrote 9 rows to search-cache-v2/csv/WQWD618.csv
LOG[INFO]: END WQWD618 (15.612388 seconds)
LOG[INFO]: START WQWH908 
LOG[INFO]: Scraped 2 rows out of 2 results
LOG[INFO]: Wrote 2 rows to search-cache-v2/csv/WQWH908.csv
LOG[INFO]: END WQWH908 (13.033150 seconds)
LOG[INFO]: START WQWV601 
LOG[INFO]: Scraped 1 rows out of 1 results
LOG[INFO]: Wrote 1 rows to search-cache-v2/csv/WQWV601.csv
LOG[INFO]: END WQWV601 (13.614211 seconds)
LOG[INFO]: START WQXB266 
LOG[INFO]: Scraped 5 rows out of 5 results
LOG[INFO]: Wrote 5 rows to search-cache-v2/csv/WQXB266.csv
LOG[INFO]: END WQ

[358,
 408,
 409,
 410,
 411,
 412,
 413,
 414,
 415,
 416,
 417,
 418,
 419,
 420,
 773,
 1574,
 1575,
 1576,
 1577,
 1578,
 1579,
 1596,
 1931,
 2300,
 2442,
 2462,
 2463,
 2464]